In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import joblib

print("Notebook chargé !")

Notebook chargé !


In [3]:
# --- Cell 2: Charger dataset plantes ---
plants = pd.read_csv("../data/plants_info.csv")
plants

,plant_id,plant_name,min_temp,max_temp,min_humidity,max_humidity,min_light,max_light
0,1,Aloe Vera,18,30,20,50,200,800
1,2,Basil,15,28,40,70,300,900
2,3,Cactus,20,40,10,30,500,1200
3,4,Lavender,15,25,30,50,600,1000
4,5,Mint,13,25,60,80,300,700
5,6,Orchid,18,27,50,80,100,500
6,7,Snake Plant,15,30,30,70,100,800
7,8,Tomato,18,30,50,80,400,1000
8,9,Rose,15,26,40,70,500,1200
9,10,Fern,18,24,70,90,200,500


In [4]:
data = []

for _, row in plants.iterrows():
    for i in range(300):  # 300 exemples par plante
        temp = np.random.randint(10, 45)
        humidity = np.random.randint(0, 100)
        light = np.random.randint(0, 1500)

        # Condition d'arrosage :
        # On doit arroser si humidité < min_humidity
        label = 1 if humidity < row["min_humidity"] else 0

        data.append([
            row["plant_id"],
            temp, humidity, light,
            label
        ])
        
df = pd.DataFrame(data, columns=["plant_id", "temp", "humidity", "light", "water"])
df.head()

,plant_id,temp,humidity,light,water
0,1,25,78,453,0
1,1,18,80,57,0
2,1,21,99,243,0
3,1,44,86,257,0
4,1,44,75,595,0


In [5]:
# --- Cell 4: Entraîner le modèle ---
from sklearn.model_selection import train_test_split

X = df[["plant_id", "temp", "humidity", "light"]]
y = df["water"]

model = DecisionTreeClassifier()
model.fit(X, y)

print("Model trained!")


Model trained!


In [6]:
# --- Cell 5: Sauvegarder le modèle ---
joblib.dump(model, "../models/watering_model.pkl")
print("Model saved as watering_model.pkl")


Model saved as watering_model.pkl


In [7]:
%pip install skl2onnx onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [8]:
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import joblib
import numpy as np

model = joblib.load("../models/watering_model.pkl")

# Forcer tous les inputs en float32
initial_type = [('input', FloatTensorType([None, 4]))]

onnx_model = convert_sklearn(model, initial_types=initial_type)

with open("../models/watering_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("ONNX OK!")


ONNX OK!


In [9]:
# --- Cell 6: Convert model to ONNX ---
import joblib
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

# Charger le modèle .pkl
model = joblib.load("../models/watering_model.pkl")

# Définir la forme d'entrée (4 features : plant_id, temp, humidity, light)
initial_type = [('input', FloatTensorType([None, 4]))]

onnx_model = convert_sklearn(model, initial_types=initial_type)

# Sauvegarder le modèle ONNX
onnx_path = "../models/watering_model.onnx"
with open(onnx_path, "wb") as f:
    f.write(onnx_model.SerializeToString())

print("✅ Model converted to ONNX:", onnx_path)


✅ Model converted to ONNX: ../models/watering_model.onnx


In [10]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("../models/watering_model.onnx")

input_name = session.get_inputs()[0].name

# Exemple: plant 1, 25°C, 30% humidité, 400 lux
test_input = np.array([[1, 25, 30, 400]], dtype=np.float32)

output = session.run(None, {input_name: test_input})
print("Pred:", output)


Pred: [array([0]), [{0: 1.0, 1: 0.0}]]
